In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [2]:
data_dir = tools.select_data_dir()
device = torch.device('cpu')

The data directory is ../data


In [3]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [4]:
# Print one sample from the dataset

x = trainset[79999]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=943, item_id=1188, rating=3


In [5]:
len(trainset)

80000

In [84]:
# Model
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        # YOUR CODE HERE
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, 20,sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, 20,sparse=True)
     
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        # YOUR CODE HERE
        return (self.user_factors(user_ids)*self.item_factors(item_ids)).sum(1)
       


In [92]:
bs = 1000
train_dataloader =trainset
test_dataloader =testset
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_biases = torch.nn.Embedding(n_users, 1)
        self.item_biases = torch.nn.Embedding(n_items,1)
        torch.nn.init.xavier_uniform_(self.user_factors.weight)
        torch.nn.init.xavier_uniform_(self.item_factors.weight)
        self.user_biases.weight.data.fill_(0.)
        self.item_biases.weight.data.fill_(0.)
        
    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item)
        pred += (self.user_factors(user) * self.item_factors(item)).sum(1, keepdim=True)
        return pred.squeeze()
nfactor = 100
model = MatrixFactorization(n, m, n_factors=nfactor)
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
loss_func = torch.nn.MSELoss()
model.to(dev)
epoches = 1000
for epoch in range(0,epoches):
    pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))  # progress bar
    count = 0
    cum_loss = 0.
    for i,( train_batch, label_batch) in pbar:
        count = 1 + i
        # Predict and calculate loss for user factor and bias
        optimizer = torch.optim.SGD([model.user_biases.weight,model.user_factors.weight], lr=0.01, weight_decay=1e-5) # learning rate
        prediction = model(train_batch[:,0].to(dev), train_batch[:,1].to(dev))
        loss = loss_func(prediction, label_batch.to(dev))    
        # Backpropagate
        loss.backward()
        
        # Update the parameters
        optimizer.step()
        optimizer.zero_grad()
        
        #predict and calculate loss for item factor and bias
        optimizer = torch.optim.SGD([model.item_biases.weight,model.item_factors.weight], lr=0.01, weight_decay=1e-5) # learning rate
        prediction = model(train_batch[:,0].to(dev), train_batch[:,1].to(dev))
        loss = loss_func(prediction, label_batch.to(dev))
        # Backpropagate
        loss.backward()
        
        # Update the parameters
        optimizer.step()
        optimizer.zero_grad()
        cum_loss += loss.item()
        pbar.set_description('training loss at {} batch {}: {}'.format(epoch,i,loss.item()))
train_loss = cum_loss/count
pbar = tqdm(enumerate(test_dataloader), total=len(test_dataloader))  # progress bar
cum_loss = 0
count = 0
for i,( test_batch, label_batch) in pbar:
    count = 1 + i
    with torch.no_grad():
        prediction = model(test_batch[:,0].to(dev), test_batch[:,1].to(dev))
        loss = loss_func(prediction, label_batch.to(dev))
        cum_loss += loss.item()
        pbar.set_description('test loss at {} batch {}: {}'.format(epoch,i,loss.item()))
test_loss = cum_loss/count
print('avg training loss: ', train_loss, ' avg test loss: ',test_loss)

NameError: name 'n' is not defined

In [85]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [54]:
# model = RecommenderSystem(n_users, n_items)

In [55]:
loss_fn = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(),lr=1e-6)

In [59]:
trainset.n_items


1682

In [67]:
user_ids,item_ids, labels = testset[:][0].to(device), testset[:][1].to(device), testset[:][2].to(device)
outputs = model.forward(user_ids, item_ids) 

IndexError: index out of range in self

In [64]:
# for epoch in range(10000):
# for user_id,item_id,rating in trainset:
model = RecommenderSystem(trainset.n_users, trainset.n_items)
batch_size = 10
user_ids = torch.arange(1, batch_size+1)
item_ids = torch.arange(1, batch_size+1)
rating=torch.arange(1, batch_size+1)
prediction = model(user_ids, item_ids)
loss = loss_fn(prediction, rating)
loss.backward()
# optimizer.step()

RuntimeError: Found dtype Long but expected Float

In [50]:
# for epoch in range(100):
# for user_id,item_id,rating in trainset:
#     prediction = model(user_id, item_id)
#     loss = loss_fn(prediction, rating)
#     loss.backward()
#     optimizer.step()
